In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
sub_paths =[
    '/opt/ml/segmentation/moon/submission/OCR_swinB_DS-ASPP_light_TTA.csv',
    '/opt/ml/segmentation/moon/submission/pseudo_deeplabv3p_swinB_TTA.csv',
    '/opt/ml/segmentation/moon/submission/OCR_swinB_TTA.csv',
]

MODEL = '+'.join(['OCR_swinB_DS-ASPP_light','pseudo_deeplabv3p_swinB','OCR_swinB'])

In [3]:
ensemble_sub = pd.read_csv('/opt/ml/segmentation/baseline_code/submission/sample_submission.csv', index_col=None)
submissions = [pd.read_csv(sub_path) for sub_path in sub_paths]
weights = [0.688,0.701,0.682] # LB scores
assert len(weights)==len(submissions)

In [4]:
for subs in tqdm(zip(*[submissions[i].iterrows() for i in range(len(submissions))])):
    subs = [(subs[j][1]) for j in range(len(subs))]
    preds = np.array([list(map(int,subs[j]['PredictionString'].split())) for j in range(len(subs))])
    file_name = subs[0]['image_id']
    
    count_map = np.zeros((11,256*256))

    ## 1. count_map : 각 pixel별로 순회하면서 어떤 label인지를 카운팅, 이 때 weight 적용
    ## 2. 각 pixel별로 가장 많이 counting된 label을 저장
    for pixel in range(256*256):
        pixel_preds = preds[:,pixel]
        for idx,pixel_pred in enumerate(pixel_preds):
            count_map[pixel_pred,pixel]+= weights[idx]*100
    
    ensemble_pred = count_map.argmax(axis=0).tolist()

    ensemble_sub = ensemble_sub.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in ensemble_pred)}, 
                                   ignore_index=True)
                                   
ensemble_sub.to_csv(f"./submission/ensemble_{MODEL}_weighted.csv", index=False)

819it [03:35,  3.79it/s]
